In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
import matplotlib.image as Image
import keras
%matplotlib inline
import bcolz
from scipy.misc import imresize
from datetime import datetime, timedelta
import h5py as h5py
try:
    utils
except:
    import utils
else:    
    reload(utils)
    print 'utils has been reloaded'

utils has been reloaded


In [73]:
# EMBEDDING CLASS TO BE USED
class Embedding:
    def __init__(self, topology='vgg16'):
        self.topology = topology
        self.network = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet',
                                                      input_tensor=None,input_shape=None,
                                                      pooling=None, classes=1000)
        self.network.layers.pop() # Get rid of the classification layer
        self.network.outputs = [self.network.layers[-1].output]
        self.network.layers[-1].outbound_nodes = []
        for layer in self.network.layers: layer.trainable=False
    def get_vector(self, batch):
        return self.network.predict(batch)

In [ ]:
API_KEY = 'J9rmStnqj0ZhQFObVxh81O84hb7UQZjY'
N_FILES = 100
SEARCH_TERM = 'cat'
api_path = "http://api.giphy.com/v1/gifs/search?q={}&api_key={}&limit={}".format(SEARCH_TERM, API_KEY, N_FILES)
data = json.loads(urllib.urlopen(api_path).read())
utils.save_files(data,SEARCH_TERM)

In [ ]:
import os
for gif_file in os.listdir('gifs'):
    gif_file_path 'gifs/'+gif_file

In [ ]:
# Pre-process gif to series of images which can be 
# fed to netwok.
gif_file = 'cat_93.gif'
gif_file_path = 'gifs/'+gif_file
frames = utils.processImage(gif_file_path,
                            reshape_to_vgg=True,
                            image_limit=20)
print np.shape(frames)
pyplot.imshow(frames[0])

In [ ]:
embeddings = Embedding()

In [32]:
# start_time = datetime.now()
embedding_matrix = embeddings.get_vector(frames)
# round(timedelta.total_seconds(datetime.now()-start_time)/60,2)

0.19

In [43]:
embedding_matrix[0:11,:].shape

(11, 4096)

In [47]:
embedding_matrix.shape
# in numpy slicing, last element is exclusive
# ex x[0:2,:] = x[[0,1],:]
max_index = embedding_matrix.shape[0]
seq_length = 11 # 10 for training, 1 for the target
data_arr = []
for i in range(max_index-seq_length):
    start_ix = i
    end_ix = i+seq_length
    target_ix = end_ix
    data_arr.append(embedding_matrix[start_ix:end_ix,:])

In [48]:
np.array(data_arr).shape

(9, 11, 4096)

In [49]:
c = bcolz.carray(data_arr, rootdir='bcolz_test2')
c.flush()

In [51]:
c.append(data_arr)
c.flush()

In [55]:
c[0:3,:,:].shape

(3, 11, 4096)

In [56]:
d = bcolz.carray(rootdir='bcolz_test2')


In [71]:
d.append(data_arr)
d[[0,2,6],:,0:10].shape,d[[0,2,6],:,10:11].shape

((3, 10, 4096), (3, 1, 4096))